In [1]:
import os
import torch

In [2]:
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    use_cuda = True
    print(f"Using CUDA.")

    print(f'_CUDA version: ')
    !nvcc --version
    print('*'*10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
else:
    device = torch.device('cpu')
    use_cuda = False
    print(f"Using CPU.")

PyTorch version: 2.6.0+cpu
**********
Using CPU.


In [3]:
from torch.utils.data import DataLoader
from torchvision import transforms

import matplotlib.pyplot as plt

from utils.final_cnn import CNN_BINARY_small
from utils.data_prep import load_file, data_preparation, plot_derivative, write_class_list
from utils.final_utils_binary import make_predictions, get_conv_layers, plot_feature_maps, plot_batch_features
from utils.final_dataset import SingleClassCustomImageDataset

In [4]:
input_name = "PPiezo"
input_file = os.path.join(os.path.join(os.getcwd(), "input"), "PPiezo_UO5.xlsx")
output_directory = os.path.join(os.path.join(os.getcwd(), 'prediction_data'), 'data')
output_image_file = os.path.join(output_directory, f'{input_name}.png')
output_class_list = os.path.join(os.path.join(os.getcwd(), 'prediction_data'), f'{input_name}_class_list.csv')
results_directory = os.path.join(os.getcwd(), 'output')
raw_data = load_file(input_file)

In [5]:
derivative_data = data_preparation(raw_data=raw_data)

In [6]:
plot_derivative(derivative_data, output_image_file)
write_class_list([input_name, 1], output_class_list)

C:\Users\2009038981\Code\DLDML\utils\data_prep.py:108: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend().set_visible(False)


In [7]:
# Define the image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((120,160)),
    transforms.Grayscale(1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,) ,(0.5,))
])

In [8]:
batch_size = 16
# Define the datasets
gp_dataset = SingleClassCustomImageDataset(output_class_list, output_directory, transform=transform)
# Load the data in batches
gp_dataloader = DataLoader(gp_dataset, batch_size=batch_size, shuffle=True)

In [9]:
model = torch.load('230912_SGD_Default.pth', weights_only=False, map_location=device)
model.load_state_dict(torch.load('230912_SGD_Default_weights.pth', map_location=device))
model.to(device)

CNN_BINARY_small(
  (conv1): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout2d(p=0.0, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout2d(p=0.0, inplace=False)
  )
  (out): Linear(in_features=96, out_features=1, bias=True)
  (sigm): Sigmoid()
)

In [10]:
make_predictions(input_model=model, input_dataloader=gp_dataloader, device=device)

[array([1.], dtype=float32)]


In [11]:
conv_layers = get_conv_layers(model_children=list(model.children()))
# Get the list of class options
classes_list = [f"Fracture absent", f"Fracture present"]

In [12]:
fig_real_batch = plot_feature_maps(input_dataloader=gp_dataloader, conv_layers=conv_layers, device=device)
plt.savefig(os.path.join(results_directory, f"{input_name}_activations.png"))
plt.close(fig_real_batch)